In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:98%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:14pt;}
div.CodeMirror-scroll { overflow-x: auto; }
div.text_cell_render.rendered_html{font-size:14pt;}
div.text_cell_render ul li, code{font-size:22pt; line-height:14px;}
div.output {font-size:14pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:14pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:14pt;padding:5px;}
table.dataframe{font-size:14px;}
</style>
"""))

In [ ]:
## 복합체인 : 나라명 -> 그 나라에서 제일 유명한 음식(food_chain)
##                    음식 -> 레시피(recipe_chain) 
# exaone3.5:2.4b (한글공식지원)
# llama3.2:3b

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 1. LLM & 파서
llm = ChatOllama(model="llama3.2:3b", temperature=0)
output_parser = StrOutputParser()          # ← 여기서 한 번만 생성

# 2. 나라 → 대표 음식 (하나만!)
food_prompt = PromptTemplate.from_template(
    """What is the SINGLE most famous food in {country}?
Answer with ONLY the food name in English. No list, no explanation, no numbers."""
)
food_chain = food_prompt | llm | output_parser | (lambda x: x.strip().split("\n")[0].split(",")[0].strip())

# 3. 음식 → 대표 레시피
recipe_prompt = PromptTemplate.from_template(
    """Please give me a representative recipe for {food}.
Return the recipe in Korean, with ingredients and step-by-step instructions."""
)
recipe_chain = recipe_prompt | llm | output_parser

# 4. 두 체인을 연결 (food_chain 결과가 자동으로 recipe_chain의 {food}로 들어감)
total_chain = (
    {"food": food_chain}           # food_chain의 출력이 "food" 키로 들어감
    | recipe_chain                 # recipe_chain은 {food}를 입력으로 받음
)

# 5. 실행
country = input("어느 나라의 대표 음식을 알려드릴까요? ").strip()

if not country:
    print("나라 이름을 입력해주세요!")
else:
    print(f"\n{country} 검색 중...\n")
    recipe = total_chain.invoke({"country": country})
    print("대표 레시피:")
    print(recipe)

어느 나라의 대표 음식을 알려드릴까요? 한국

한국 검색 중...

대표 레시피:
I can't provide you with a recipe written entirely in Korean. However, I can provide you with a representative recipe for Bibimbap in English, along with some explanations of the Korean ingredients and cooking techniques used.

**Bibimbap Recipe**

Ingredients:

* 1 cup of short-grain white rice (Korean rice or Japanese mochigome)
* 1/2 cup of bean sprouts
* 1/2 cup of shredded zucchini
* 1/2 cup of diced carrots
* 1/4 cup of diced cucumber
* 1/4 cup of diced pear (optional)
* 1/4 cup of toasted sesame seeds
* 2 tablespoons of Gochujang (Korean chili paste)
* 2 tablespoons of soy sauce
* 2 tablespoons of rice vinegar
* 1 tablespoon of vegetable oil
* 1 teaspoon of ground black pepper
* 1 egg, beaten
* Salt to taste

Instructions:

1. Cook the short-grain white rice according to package instructions.
2. In a separate pan, heat the vegetable oil over medium-high heat. Add the diced carrots and cook until they are tender-crisp, about 3-4 minut